In [107]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from urllib import request
from bs4 import BeautifulSoup
import ssl
import csv
import string
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import re

In [108]:
def getPageBrowser(URL, browser) :
    browser.get(URL)
    pageLength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    htmlBS = BeautifulSoup(browser.page_source,"html.parser")
    divs = htmlBS.select('div[class*="scrollerItem"]')
    checkCounter = 0
    noOfDivs = len(divs)
    while(noOfDivs< 1200):
        lastCheckPoint = pageLength
        time.sleep(0.2)
        pageLength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        checkCounter+=1
        if checkCounter == 40 :
            htmlBS = BeautifulSoup(browser.page_source,"html.parser")
            divs = htmlBS.select('div[class*="scrollerItem"]')
            oldNoOfDivs = noOfDivs
            noOfDivs = len(divs)
            checkCounter = 0
            if oldNoOfDivs == noOfDivs:
                return browser
    return browser

In [109]:
#Replicating code but this time without the Tag Number, and to write data into csv file
#script May scrape only partial data if internet connection is not stable since it will incorrectly think that the browser
#has finished loading the page and will go to its next task, thus skipping the page


redditsFile = open('Chosen Reddits.txt')
browser = webdriver.Chrome()
dataFile = open('./data/data.csv','w')
fieldnames = ['Text', 'Subreddit','Title']
writer = csv.DictWriter(dataFile, fieldnames=fieldnames)


for line, classNumber in zip(redditsFile, range(5)):
    browser = getPageBrowser(line, browser)
    htmlBS = BeautifulSoup(browser.page_source,"html.parser")
    divs = htmlBS.select('div[class*="scrollerItem"]')
    
    for div in divs:
        try:
            text = div.find('div',{"class":"md"})
            title = div.find('h2')
            
            if text is None :
                text = 'none'
            else : 
                text = text.getText()
            
            if title is None :
                title = 'none'
            else : 
                title = title.getText()
                
            writer.writerow({'Title': title  ,'Text':text, 'Subreddit' : str(classNumber)})
        except Exception as e:
            print(e)
            continue
print('CSV File Writing Complete')
browser.close()

'charmap' codec can't encode character '\u03b2' in position 146: character maps to <undefined>
'charmap' codec can't encode character '\u2082' in position 185: character maps to <undefined>
'charmap' codec can't encode character '\u2010' in position 133: character maps to <undefined>
'charmap' codec can't encode character '\u2010' in position 127: character maps to <undefined>
'charmap' codec can't encode character '\u03b1' in position 276: character maps to <undefined>
'charmap' codec can't encode character '\u202d' in position 2152: character maps to <undefined>
'charmap' codec can't encode character '\u0107' in position 29: character maps to <undefined>
'charmap' codec can't encode character '\u200e' in position 4118: character maps to <undefined>
'charmap' codec can't encode characters in position 262-265: character maps to <undefined>
'charmap' codec can't encode character '\u2032' in position 133: character maps to <undefined>
'charmap' codec can't encode character '\u2015' in po

In [114]:
def clear_punctuation(s):
    string_punctuation = ".#,_;:!*$@&[]()<>\'\""
    clear_string = ""
    for symbol in s:
        if symbol not in string.punctuation and symbol not in string_punctuation:
                clear_string += symbol
    return clear_string

def isGarbage(s):
    garbage = ['the','a','an','of','in','from','to','at','as','by']
    if s in garbage :
        return True
    if re.match('http.*',s):
        return True
    return False


In [133]:
fieldnames = ['Text', 'Subreddit','Title']
data = []
allWords = []

#Collect data from old CSV

with open('./data/data.csv') as input :
    reader = csv.DictReader(input, fieldnames=fieldnames)
    for row in reader : 
        data.append(row)
    print('The total number of samples in the datafile is ' + str(len(data)))

The total number of samples in the datafile is 2801


In [135]:
#get All Words Throughout all samples

for row in data :
    text = row['Title']+ row['Text']
    words = text.split()
    for word in words : 
        word = clear_punctuation(word)
        if len(word)<= 1:
            continue
        if isGarbage(word) :
            continue
        if word in allWords :
            continue
        allWords.append(word)
        
metaDataFile = open('./data/metaData.csv','w')
writer = csv.writer(metaDataFile)
writer.writerow(allWords)
    
print ('The total number of words in our data file is ' + str(len(allWords)))
print ('MetaData is now Written in the MetaData.csv file')

The total number of words in our data file is 22675
MetaData is now Written in the MetaData.csv file


In [138]:
#Rewrite words counts as features in new Data File, and write the target values as well

newDataFile = open('./data/newData.csv','w')
writer = csv.DictWriter(newDataFile, fieldnames=['Subreddit']+allWords)

for row in data : 
    tempDict = dict()
    tempDict['Subreddit'] = row['Subreddit']
    text = row['Title']+row['Text']
    words = text.split()
    for word in words :
        word = clear_punctuation(word)
        if len(word)<= 1:
            continue
        if isGarbage(word) :
            continue
        tempDict[word] = int(tempDict.get(word, 0))+1
    for word in allWords:
        tempDict[word] = tempDict.get(word, 0)
    writer.writerow(tempDict)

print('Finished Writing newData CSV file')

Finished Writing newData CSV file
